# Data Cleaning

Poniższy notebook sporządzono na podstawie kursu dostępnego pod adresem https://www.kaggle.com/learn/data-cleaning

**Zawiera on jedynie wybrane zagadnienia!**

Agenda:
1. **Handling Missing Values**
2. **Scaling and Normalization**
3. **Parsing Dates**
4. Character Encodings (brak poniżej, więcej: https://www.kaggle.com/alexisbcook/character-encodings)
5. **Inconsistent Data Entry**

## 1) HANDLING MISSING VALUES

### Rzucenie okiem na dane

Pierwszą czynnością pracy z danymi jest ich wczytanie i wstępna (wizualna) analiza:
- dataset.head()
- dataset.describe()

Warto również zapoznać się z **dokumentacją** zbioru danych, jeśli korzystamy ze zbioru utworzonego przez kogoś innego (np. z kaggle.com).

### Określenie liczby brakujących danych (NaN, None)

Dobrą praktyką jest sprawdzenie czy nasz zbiór danych posiada jakieś braki:

<img src="Images/img_22.jpg">

Już po pierwszych 10 kolumnach widać, że mamy sporo brakujących wartości. Aby uchwycić skalę problemu najlepiej jest określić **procent** brakujących wartości w naszym zbiorze danych.

<img src="Images/img_23.jpg">

Widać wyraźnie, że blisko 1/4 komórek jest pusta (NaN lub None).

### Określenie skąd się biorą braki danych

_"Look at your data and try to figure out why it is the way it is and how that will affect your analysis"_

Aby sprawnie radzić sobie z brakami danych potrzebne jest doświadczenie (i intuicja!). Dobrym pytaniem, które warto sobie postawić próbując znaleźć odpowiedź na pytanie skąd biorą się brakujące wartości jest: 

**Czy konkretny brak danych wynika z faktu, że dana wartość nie została zarejestrowana, czy ponieważ w ogóle nie istnieje?**

Jeśli mamy do czynienia z brakiem, który wynika z faktu, że dana wartość nie istnieje (np. wzrost najstarszego dziecka kogoś, kto w ogóle nie ma dzieci), to nie ma sensu szukać sposobu na uzupełnienie takiego braku - najlepiej pozostawić wartość NaN.

Jeśli jednak brak wynika z tego, że dana wartość nie została zarejestrowana (ale prawdopodobnie może istnieć), wtedy można pokusić się o "przewidzenie" brakującej wartości w oparciu o istniejące wartości danej kolumny (cechy) - proces ten nosi nazwę **imputacji (ang. imputation)**.

Przykładowo, dla pierwszych 10 kolumn jak wyżej, kolumna _time_ określa liczbę sekund jakie pozostały do końca meczu w momencie wykonania akcji. Wynika stąd, że wszystkie braki w kolumnie _time_ wynikają z tego, że po prostu ich nie zarejestrowano! Stąd, sensowne wydaje się podjąć próbę ich zastąpienia zamiast pozostawić jako NA.

Z drugiej strony kolumna _PenalizedTeam_ również zawiera wiele braków, ale w tym przypadku biorą się one z faktu, że po prostu żadna z drużym nie została w danej akcji ukarana (nie było przewinienia), dlatego szukanie "na siłę" wartości (drużyny) do wstawienia jest bez sensu i należy pozostawić NA lub ewentualnie wstawić coś a'la "brak".

**WNIOSEK:**

Warto skupić się na każdej kolumnie z osobna i dość szczegółowo przeanalizować w jaki sposób najlepiej poradzić sobie z brakującymi wartościami.

### Usuwanie brakujących wartości

Jedną z najszybszych (ale niekoniecznie najlepszych) metod radzenia sobie z brakami danych jest usunięcie wierszy (lub nawet kolumn, jeśli braków jest ponad 50% w danej kolumnie) zawierających braki.

<img src="Images/img_24.jpg">

Jak widać, użycie metody "dropna" może spowodować usunięcie wszystkich wierszy, jeśli każdy z nich zawierał przynajmniej jedną brakującą wartość. W takiej sytuacji lepszym rozwiązaniem może być usunięcie kolumn, które posiadają braki:

<img src="Images/img_25.jpg">

<img src="Images/img_26.jpg">

Metoda polegająca na usunięciu kolumn również nie jest najlepsza, bo doprowadziła do ponad 2-krotnego zmniejszenia naszego zbioru danych...

**WSKAZÓWKA:**

Metoda całkowitego usuwania brakujących wartości nie jest rekomendowana w pracy przy ważnych komercyjnych projektach!

### Automatyczne uzupełnianie brakujących wartości

Inną opcji radzenia sobie z brakami danych jest ich uzupełnienie.

(Poniższy przykład celowo dotyczy wycinka całego zbioru danych, aby pokazać o co chodzi.)

<img src="Images/img_27.jpg">

Używając modułu Pandas mamy możliwość skorzystania z metody _fillna(n)_ , która zastępuje wszystkie braki danych NAN występujące w DataFramie wskazaną wartością _n_ .

<img src="Images/img_28.jpg">

Innym, nieco sprytniejszym, sposobem jest zastąpienie braku wartością, która występuje bezpośrednio po nim (po braku) w tej samej kolumnie. Takie działanie ma sens w przypadku zbiorów, w których obserwacje mają jakiś logiczny porządek.

<img src="Images/img_29.jpg">

## 2) SCALING AND NORMALIZATION

### Skalowanie vs. Normalizacja: Jaka jest różnica?

Jeden z powodów, dlaczego łatwo się pogubić poruszając temat skalowania i normalizacji jest fakt, że oba te terminy często są stosowane zamiennie. Dzieje się tak, ponieważ oba procesy są bardzo podobne! Zarówno jeden, jak i drugi polega na transformacji danych liczbowych (zmiennych numerycznych) w taki sposób, aby dane te po przekształceniu miały pewne, określone (i pomocne z punktu widzenia modelu) właściwości. Różnica pomiędzy skalowaniem i normalizacją polega na tym, że:
* podczas **skalowania** zmieniamy _zakres (range)_ danych
* podczas **normalizacji** zmieniamy _kształt rozkładu (shape of the distribution)_ danych

**Moduły dla skalowania i normalizacji:**

<img src="Images/img_30a.jpg">

### Skalowanie

Polega na modyfikacji naszych danych w taki sposób, że mieszczą się one w określonym zakresie, np 0-100 lub 0-1. Skalowanie jest stosowane, gdy używamy algorytmów opartych na miarach odległości między punktami danych, np. **SVM (Support Vector Machines)** lub **K-NN (K-Nearest Neighbours)**. W tych algorytmach zmiana o "1" w dowolnej funkcji numerycznej ma taką samą wagę.

Przykładowo, nasz zbiór danych może zawierać ceny niektórych produktów w jenach i dolarach. Jeden dolar to około 100 jenów, ale jeśli nie przeskalujemy cen w naszym zbiorze, to algorytmy takie jak SVM czy KNN potraktują różnicę w cenie 1 jena tak samo ważną, jak różnicę 1 dolara!!! A co jeśli mamy wzrost i wagę? Tutaj akurat nie da się jasno określić ile kg powinno równać się jednemu centymetrowi, dlatego skalowanie nie ma zastosowania.

Skalowanie umożliwa zatem porównywanie różnych zmiennych "na równych zasadach".

<img src="Images/img_31.jpg">

Zauważmy, że _kształt_ danych na powyższym przykładzie nie zmienił się. Zmianie uległ natomiast zakres wartości (oś pozioma) z 0-8 na 0-1.

### Normalizacja

W porównaniu do skalowania, normalizacja jest bardziej radykalną transformacją danych. Celem normalizacji jest zmiana obserwacji w ten sposób, aby dało się je opisać za pomocą **rozkładu normalnego**.

Ogólnie rzecz biorąc, normalizacje stosuje się, gdy zamierzamy używać modelu uczenia maszynowego lub techniki statystycznej, która zakłada, że dane posiadają rozkład normalny. Przykładami takich technik są **analiza wariancji (ANOVA)** , **regresja liniowa** , **liniowa analiza dyskryminacyjna (LDA)** , **naiwny klasyfikator Bayesowski** oraz (w ciemno) metody zawierające **"Gaussian"** w nazwie.

Jedną z metod normalizacji jest metoda **Box'a-Cox'a**:

<img src="Images/img_32.jpg">

Zauważmy, że _kształt_ danych na powyższym przykładzie uległ zmianie. Przed normalizacją był niemal w kształcie litery "L", zaś po normalizacji przypomina rozkład normalny (krzywa dzwonowa).

## 3) PARSING DATES

Poniżej, na przykładowym zbiorze danych omówiono sposoby radzenia sobie z niewłaściwymi formatami dat i ich zamianą na format rozpoznawany przez Pythona.

Początkowy format kolumny 'date':

<img src="Images/img_33.jpg">

Jak widać kolumna 'date' ma typ 'object', a więcj Python traktuje ją jak string.

### Konwersja na typ _datetime_

Proces konwersji na typ _datetime_ nosi nazwę **parsing dates** (parsowanie dat), ponieważ bierzemy string i każdy jego element identyfikujemy jako konkretną część daty. Przede wszystkim polega to na zidentyfikowaniu jaką część daty reprezentują poszczególne lementu stringa i czym są oddzielone od siebie. Najpowszechniejsze: %d - day, %m - month, %y - two-digit year, %Y - four-digit year). Przykłady:

- 1/17/07 ma format: %m/%d/%y
- 17-1-2007 ma format: %d-%m-%Y

Nasz przykład ewidentnie pasuje do formatu "%m/%d/%y". 

<img src="Images/img_34.jpg">

Nasza nowopowstała kolumna 'date_parsed' posiada typ "datetime64", który posiada porządek zapisu "year-month-day":

<img src="Images/img_35.jpg">

**Co w sytuacji, gdy otrzymamy błąd "multiple date formats"?**

Może się zdarzyć, że kolumna, którą chcemy "sparsować" na format "datetime64" zawiera stringi reprezentujące daty w kilku różnych formatach. W takiej sytuacji możemy użyć _pandas_ z, aby ten spróbował sobie z tym poradzić poprzez funkcję _"infer_datetime_format = True"_ :

<img src="Images/img_36.jpg">

**Dlaczego nie używać _"infer_datetime_format = True"_ za każdym razem?**

Dwa powody:
- nie zawsze pandas będzie w stanie poradzić sobie z rozszyfrowaniem formatu
- metoda _"infer..."_ jest dużo wolniejsza niż dokładne wskazanie formatu przez nas

### Wyciąganie elementów z daty

Gdy mamy już datę w formacie "datetime" możemy wyciągać z niej interesujące nas elementy. Poniżej np. wyciągniemy dzień miesiąca:

<img src="Images/img_37.jpg">

### Sprawdzenie czy "sparsowaliśmy" daty poprawnie

Jednym z najczęstszych błędów związanych z zamianą stringów na daty jest pomieszanie miesięcy i dni, ponieważ funkcja _"to_datetime()"_ w żaden sposób nie informuje nas czy np. dany miesiąc nie ma wartości >12, albo czy wartości dla dnia miesiąca nie kończą się na 12. Najlepszym sposobem na sprawdzenie czy wszystki przebiegło pomyślnie jest zwizualizowanie, np. w formie histogramu, wartości dla dni i sprawdzenie czy zawierają się w przedziale 1-31.

<img src="Images/img_38.jpg">

## 4) INCONSISTENT DATA ENTRY

**Przydatne moduły:**

<img src="Images/img_39.jpg">

### Zajrzenie w dane

<img src="Images/img_40.jpg">

Skupimy się na kolumnie "Country" i spróbujemy upewnić się, że nie zawiera ona żadnych niespójności, jeśli chodzi o wartości, które zawiera.

<img src="Images/img_41.jpg">

Łatwo zauważyć, że istnieją niespójności w kolumnie "Country", np. ' Germany' i 'germany'. 

### Wielkość liter i białe znaki

Pierwszą rzeczą, którą warto zrobić jest zamiana wszystkich wartości na _lowercase_ (w razie czego łatwo ten proces odwrócić) oraz pozbyć się białych znaków na początku i końcu każdej wartości. 

**Wskazówka**: Niespójności w danych tekstowych związane z wielkością liter lub występowaniem białych znaków na początku/końcu wartości są bardzo powszechne i pozbywając się ich można rozwiązać około 80% problemów dot. niespójnych danych

<img src="Images/img_42.jpg">

### Moduł _fuzzywuzzy_

Sprawdźmy czy występują jeszcze jakieś niespójności w kolumnie "Country" oprócz powyższych:

<img src="Images/img_43.jpg">

Widać, że występują jeszcze inne niespójności, np. 'south korea' i 'southkorea' czy 'usa' i 'usofa'.

Można próbować rozwiązać je ręcznie, ale jest to czasochłonne, dlatego skorzystamy z modułu **_fuzzywuzzy_** . Pozwala on zidentyfikować, które stringi są do siebie najbardziej podobne.

**Fuzzy matching:** Proces polegający na automatycznym znajdowaniu stringów, które są najbardziej podobne do wskazanego łańcucha znaków (target string). Podobieństwo w tym przydku jest rozumiane następująco: jeden string jest tym bardziej podobny do drugiego, im mniej znaków trzeba zamienić, aby oba były identyczne. Przykładowo, "apple" i "snapple" - dwie zmiany (dodanie "s" i "n" do "apple", aby otrzymać "snapple"). _Fuzzy matching_ , to nie zawsze jest to użyteczna metoda, ale ogólnie pomocna.

_Fuzzywuzzy_ zwraca stosunek (ratio) dwóch stringów. Im stosunek bliższy 100, tym mniejsza różnica między dwoma stringami. W powyższym przykładzie otrzymamy dziesięć stringów z listy miast, które mają najmniejszy dystans do "south korea":

<img src="Images/img_44.jpg">

Jak widać najbardziej podobnym stringiem do "south korea" jest "southkorea", dlatego zastąpimy wszystkie wiersze kolumny "Country", gdzie stosunek podobieństwa do "south korea" (ratio) jest > 47. Do tego celu stworzono poniższą funkcję:

<img src="Images/img_45.jpg">

Zamiana:

<img src="Images/img_46.jpg">

Sprawdzenie:

<img src="Images/img_47.jpg">